In [ ]:
!pip uninstall tensorflow -y

Found existing installation: tensorflow 2.12.0
Uninstalling tensorflow-2.12.0:
  Successfully uninstalled tensorflow-2.12.0


In [ ]:
!pip  install transformers==4.22.1 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 46.1 MB/s eta 0:00:00


In [23]:
!pip install pyttsx3

In [272]:
import pandas as pd
import re
import transformers
from transformers import pipeline, DistilBertTokenizerFast, DistilBertForSequenceClassification
import pyttsx3

In [254]:
def get_text():
    df = pd.read_json("df.json")
    test_df = df[4000:4101][['text','label']].copy()
    test_text = test_df['text'].values
    test_label = test_df['label'].values
    return test_text, test_label

In [255]:
def label_tokens(token_lists, seq):
    #Labeling as 1 only for the token, which is the last token of the setence labeled as 1
    t_l = []
    for i, t in enumerate(range(len(token_lists))):
        if seq !=0 and i == len(token_lists)-1:
            t_l.append(test_label[seq])
        else :
            t_l.append(0)
    return t_l

In [256]:
def flatten_list(nested_list):
    #Changing the nested list to a one-dimensional list express
    flat_list = []
    for sublist in nested_list:
        flat_list.extend(sublist)
    return flat_list

In [257]:
def split_token_sentences(list_sentence):
    #Splitting sentence to tokens
    #Labeling as 1 only for the token, which is the last token of the setence labeled as 1

    s = []
    t_s = []
    t_l = []
    i=0
    for seq, text in enumerate(list_sentence):
        for match in re.finditer(r"\,|\.|\;|\?|\!", text):
            a = text[i:match.start()+1]
            a.strip()
            b = a.split()
            s.append(a)
            t_s.append(b)
            if match.start() != len(text) - 1:
                i = match.start()+1
                t_l.append(label_tokens(b, 0))
            else:
                i=0
                t_l.append(label_tokens(b, seq))
    return s, t_s, t_l

In [258]:
def generate_test_dataset(tokenized_text, labels):
    #Generating sequences to generate different sequences of speaking by combining tokens
    #Labeling sequences to identify the moments when each speaker takes their turn

    test_seq_tokens= []
    test_seq_labels= []
    t=''
    s=0
    c=0
    first_1 = labels.index(1)
    for i in range(len(tokenized_text)):
        if labels[i] == 1 and c==0 :
            test_seq_tokens.append(tokenized_text[i])
            test_seq_labels.append(labels[i])
            t = ''
        elif labels[i] == 1 and c!=0 :
            t = ' '.join(tokenized_text[s:i+1])
            test_seq_tokens.append(t)
            test_seq_labels.append(labels[i])
            s = i+1
            t = ''
        elif labels[i] == 0 and c == 0:
            if i == 0:
                t = ' '.join(tokenized_text[s:i+1])
                test_seq_tokens.append(t)
                test_seq_labels.append(labels[i])
            s = i
            c = 1
        elif labels[i] == 0 and c != 0:
            t = ' '.join(tokenized_text[s:i+1])
            test_seq_tokens.append(t)
            test_seq_labels.append(labels[i])
        else:# labels[i] == 1 and c != 0:
            t = ' '.join(tokenized_text[s:i+1])
            test_seq_tokens.append(t)
            test_seq_labels.append(labels[i])
            s = 0
            c = 0
            t = ''
    return test_seq_tokens, test_seq_labels

In [294]:
def generat_voice(text, pred):
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[0].id)
    engine.setProperty('rate', 200)
    turn = ''
    v=0
    for i in range(len(text)):
        turn = turn+text[i]+' '
        if pred[i]==1:
            speaker = ["A" if v == 0 else "B"]
            print(speaker, end = ' ')
            print(turn)
            engine.say(turn)
            engine.runAndWait()
            turn = ''
            v = 1 if v == 0 else 0 
            engine.setProperty('voice', voices[v].id)

In [296]:
if __name__ == '__main__':
    test_text, test_label = get_text()
    text, tokenized_text, labels = split_token_sentences(test_text)
    label = flatten_list(labels)
    tokenized_text = flatten_list(tokenized_text)
    sequences, labels = generate_test_dataset(tokenized_text, label)
    tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased', ignore_mismatched_sizes=True)
    pipe = pipeline("text-classification", './model/bert_distilbert_seq_token', tokenizer=tokenizer)
    
    prediction = []
    for t in sequences[:100]:
        print(t, end='  ')
        p_l = pipe(t)[0]['label']
        prediction.append(p_l)
        print([p_l])
        
    text = tokenized_text[:100]
    v_pred = [0 if x == 'Continue' else 1 for x in prediction]
    
    
    import time
    
    time.sleep(5)
    print("\n\n\nVoice Generating")
    
    generat_voice(text, v_pred)

Hold,  ['Continue']
Hold, Just  ['Continue']
Hold, Just a  ['Continue']
Hold, Just a moment.  ['Continue']
Hold, Just a moment. I  ['Continue']
Hold, Just a moment. I have  ['Continue']
Hold, Just a moment. I have located  ['Continue']
Hold, Just a moment. I have located a  ['Continue']
Hold, Just a moment. I have located a few  ['Continue']
Hold, Just a moment. I have located a few hotels,  ['Continue']
Hold, Just a moment. I have located a few hotels, The  ['Continue']
Hold, Just a moment. I have located a few hotels, The Night  ['Continue']
Hold, Just a moment. I have located a few hotels, The Night Times  ['Continue']
Hold, Just a moment. I have located a few hotels, The Night Times Square  ['Continue']
Hold, Just a moment. I have located a few hotels, The Night Times Square and  ['Continue']
Hold, Just a moment. I have located a few hotels, The Night Times Square and Double  ['Continue']
Hold, Just a moment. I have located a few hotels, The Night Times Square and Double Tree  ['Co